In [1]:
!nvidia-smi

Sun Jul 12 20:46:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:65:00.0  On |                  N/A |
| 23%   40C    P0    58W / 250W |    790MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
import pandas as pd

In [4]:
def load_result_table(data_path):
    df = pd.read_csv(data_path, encoding='utf-8')
    #tmp = tmp[:10]
    df = df.rename(columns={"tags_result":"tags", "songs_result":"songs"})
    df = df.drop("Unnamed: 0", axis=1)
    df['tags'] = df["tags"].map(lambda x : eval(x))
    df['songs'] = df["songs"].map(lambda x : eval(x))
    
    return df

안녕하세요 카카오 아레나입니다.

1. zip 포멧으로 결과파일을 압축되어 제출 되어야함
2. 압축푼 파일명은 results.json 이름으로 1개만 있어야함
3. 각 플레이리스트 추천 결과에서 songs 100개, tags 10개 여야함
4. 각 플레이리스트 추천 결과에서 songs와 tags의 값은 중복없이 유일해야함
5. val.json의 플레이리스트ID들과 제출물의 플레이리스트ID 23015개가 정확히 일치되어야함



In [37]:
s2p2v_result = load_result_table("base_list.csv")
t2p2v_result = load_result_table("t2p2v_rec_model_result.csv")

In [11]:
t2p2v_result.head()

,id,tags,songs
0,118598,[],[]
1,131447,"[발라드, 힙합, 기분전환, 락, 랩, 설렘, 댄스, 사랑, 팝, 휴식]","[476680, 253594, 688951, 550695, 38832, 520620..."
2,51464,[],[]
3,45144,[],[]
4,79929,[],[]


In [51]:
s2p2v_result['tags_len'] = s2p2v_result["tags"].map(lambda x : len(x))
s2p2v_result['songs_len'] = s2p2v_result["songs"].map(lambda x : len(x))

In [26]:
## s2v결과 중 tag, song의 개수가 부족한 경우, t2v에서 데이터 가져오기

def emsemble(idx, data, isTag=True):
    if isTag ==True:
        target = 10
        col = "tags"
    else:
        target =100
        col = "songs"
        
    if len(data)< target:
        add_list = list(t2p2v_result[t2p2v_result["id"]==idx][col])[0]
        add_list = [x for x in add_list if x not in data]

        data = data + add_list
        if len(data) > target:
            data = data[:target]

    return data

In [40]:
tmp  = s2p2v_result[s2p2v_result["songs_len"]==7][:10]

In [50]:
s2p2v_result["tags"] = s2p2v_result.apply(lambda x : emsemble(x['id'], x['tags'], True), axis=1)
s2p2v_result["songs"] = s2p2v_result.apply(lambda x : emsemble(x['id'], x['songs'], False), axis=1)

In [58]:
tag_list = list(s2p2v_result["tags"])

flat_tag_list = []
for i in tag_list:
    for j in i:
        flat_tag_list.append(j)
        
tag_list = list(s2p2v_result["songs"])

flat_song_list = []
for i in tag_list:
    for j in i:
        flat_song_list.append(j)

In [59]:
from collections import Counter

def get_count_list(token_list):
    count_list = sorted(dict(Counter(token_list)).items(), key=(lambda x :x[1]), reverse=True)
    return [x for x in count_list if x[1]>1]

In [61]:
most_song = [x[0] for x in get_count_list(flat_song_list)[:200]]
most_tag = [x[0] for x in get_count_list(flat_tag_list)[:20]]

In [64]:
def adj_song_result(songs):
    songs = list(set(songs))
    if len(songs)<100:
        tmp_most_song = [x for x in most_song if x not in songs]
        songs = songs + tmp_most_song
        songs = songs[:100]
        
    else:
        pass
    
    return songs

def adj_tag_result(tags):
    tags = list(set(tags))
    if len(tags)<10:
        tmp_most_tag = [x for x in most_tag if x not in tags]
        tags = tags + tmp_most_tag
        tags = tags[:10]
        
    else:
        pass
    
    return tags

In [65]:
s2p2v_result["tags"] = s2p2v_result['tags'].map(lambda x : adj_tag_result(x))
s2p2v_result["songs"] = s2p2v_result['songs'].map(lambda x : adj_song_result(x))

In [68]:
answers = []
for idx_, tags, songs in zip(s2p2v_result["id"], s2p2v_result["tags"], s2p2v_result["songs"]):
    answers.append({
        "id":idx_,
        "songs": songs,
        "tags": tags,
    })

In [69]:
def write_json(data, fname):
    def _conv(o):
        if isinstance(o, (np.int64, np.int32)):
            return int(o)
        raise TypeError

    with io.open(fname, "w", encoding="utf-8") as f:
        json_str = json.dumps(data, ensure_ascii=False, default=_conv)
        f.write(json_str)

In [71]:
import os, io
import json

write_json(answers, "results.json")

In [137]:
loaded_data = []
with open("result.json", "r", encoding="utf-8") as data_file:
    for line in data_file:
        doc_obj=ujson.loads(line.strip())
        loaded_data.append(doc_obj)
    data = pd.DataFrame.from_dict(loaded_data)